In [2]:
## Import the snowflake libraries
from snowflake.snowpark.session import Session

In [3]:
## Connect to your snowflake Account

def create_session_object():
    connection_paramters = {
        "account":"mysnfAccount",
        "user":"rpegu",
        "password":"******",
        "role":"demo_role",
        "warehouse":"demo_wh",
        "database":"snf_exntac",
        "schema" :"api_data"
    }

    session = Session.builder.configs(connection_paramters).create()
    return session 

In [4]:
session = create_session_object()

In [5]:
# Create a internal Stage (option step- you can use any existing stage)
session.sql("create or replace stage stage_ftp").collect()

[Row(status='Stage area STAGE_FTP successfully created.')]

In [8]:
## Create the python function to read the ftp files
import tempfile
import os
from ftplib import FTP
def read_ftp_transfer(Ftp_address,ftp_directory,stage_name,snf_session):
    ftp =FTP(Ftp_address)# get the ftp address
    #login to ftp 
    ftp.login()
    ftp.cwd(ftp_directory)
    filenames = ftp.nlst()
    with tempfile.TemporaryDirectory() as td:
        count = 0
        for filename in filenames:
            if filename.endswith('tsv.gz'): # can be any extension like tsv.gz or whatever file you wish get
                tmp_file = os.path.join(td,filename)
                #print(filename)
                #transfering the file into snowflake stage
                with open(tmp_file,'wb') as fh:
                    ftp.retrbinary('RETR ' + filename, fh.write)
                    snf_session.file.put(tmp_file,stage_name)
                    count +=1
    msg = "Number of file transfer: " + str(count)
    return(msg)

In [9]:
ftp_address ='ftp.ebi.ac.uk'
ftp_directory = 'pub/databases/gwas/summary_statistics/GCST90000001-GCST90001000/GCST90000014/harmonised'
stg_name ='@stage_ftp'


In [10]:
read_ftp_transfer(ftp_address,ftp_directory,stg_name,session)

'Number of file transfer: 2'